In [ ]:
!pip uninstall tensorflow tensorflow-estimator tensorflow-io-gcs-filesystem keras --yes

In [ ]:
!pip cache purge

In [ ]:
!pip install tensorflow==2.15.0

In [1]:
import tensorflow as tf
import keras

print(keras.__version__)
print(tf.__version__)

2024-03-24 11:51:09.338404: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-03-24 11:51:09.338514: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-03-24 11:51:09.340335: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


2.15.0
2.15.0


In [ ]:

import matplotlib.pyplot as plt
import os
import zipfile
import math
import shutil
import glob
import numpy as np
import warnings
warnings.filterwarnings('ignore')

from PIL import Image

In [ ]:
!cp -r /kaggle/input/brian-tumor-dataset /kaggle/working

In [ ]:
import os
import numpy as np
import math
import shutil

def create_data_directory(DEST_ROOT, ROOT_DIR, type='train', size=1):
    """
    This function creates the directory for the given type of data, like train, test and validation.
    :param DEST_ROOT: The destination root directory of the data, usually the 'data' directory.
    :param ROOT_DIR: The root directory of the data.
    :param type: The type of data like train, test and validation.
    :param size: The size of the data in the directory like 0.7, 0.2 and 0.1.
    :return: A directory for the given type of data.
    """
    if not os.path.exists(os.path.join(ROOT_DIR)):
        print(f"The source directory {ROOT_DIR} does not exist.")
        return

    number_of_images = {}

    for i in os.listdir(ROOT_DIR):
        number_of_images[i] = len(os.listdir(os.path.join(ROOT_DIR, i)))

    dest_dir = os.path.join(DEST_ROOT, type)

    if not os.path.exists(dest_dir):
        try:
            os.makedirs(dest_dir)
        except PermissionError:
            print(f"Permission denied: Could not create directory {dest_dir}")
            return

        for dir in os.listdir(ROOT_DIR):
            new_dir = os.path.join(dest_dir, dir)

            try:
                os.makedirs(new_dir)
            except PermissionError:
                print(f"Permission denied: Could not create directory {new_dir}")
                return

            images = os.listdir(os.path.join(ROOT_DIR, dir))
            np.random.shuffle(images)

            # Get the number of images for the given type of data as per the size parameter..
            size_dir = math.floor(number_of_images[dir] * size)

            # Move the images to the given type of data directory.
            for image in images[:size_dir]:
                src = os.path.join(ROOT_DIR, dir, image)
                dest = os.path.join(new_dir, image)

                if os.path.exists(src) and not os.path.exists(dest):
                    try:
                        shutil.move(src, dest)
                    except PermissionError:
                        print(f"Permission denied: Could not move file {src} to {dest}")
                        return
                else:
                    print(f"File {src} does not exist or file {dest} already exists.")
    else:
        print(f"The directory {type} already exists in {DEST_ROOT}")

In [ ]:
create_data_directory('data', "/kaggle/working/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set", "test", 0.15)
print(f"The test directory has been created with {len(os.listdir('data/test'))} subdirectories.")
create_data_directory('data', "/kaggle/working/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set", "val", 0.15)
print(f"The val directory has been created with {len(os.listdir('data/val'))} subdirectories.")
create_data_directory('data', "/kaggle/working/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set", "train")
print(f"The train directory has been created with {len(os.listdir('data/train'))} subdirectories.")

In [ ]:
len(os.listdir('data/train/Brain Tumor')) + len(os.listdir('data/train/Healthy'))

In [ ]:
len(os.listdir('data/val/Brain Tumor')) + len(os.listdir('data/val/Healthy'))

In [ ]:
len(os.listdir('data/test/Brain Tumor')) + len(os.listdir('data/test/Healthy'))

In [ ]:
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense, BatchNormalization, Dropout, GlobalAvgPool2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import tensorflow.keras as keras

In [ ]:
strategy = tf.distribute.MirroredStrategy()

In [ ]:
with strategy.scope():
    model = Sequential()

    model.add(Conv2D(filters=16,kernel_size=(3,3),activation="relu",input_shape=(224,224,3), padding='same'))

    model.add(Conv2D(filters=32,kernel_size=(3,3),activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(filters=64,kernel_size=(3,3),activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Conv2D(filters=128,kernel_size=(3,3),activation="relu"))
    model.add(BatchNormalization())
    model.add(MaxPool2D(pool_size=(2,2)))

    model.add(Dropout(rate=0.25))

    model.add(Flatten())
    model.add(Dense(units=64,activation='relu'))
    model.add(Dropout(rate=0.25))
    model.add(Dense(units=1,activation='sigmoid'))
    
    # Use AdamW optimizer
    model.compile(optimizer=tf.keras.optimizers.AdamW(learning_rate=1e-4),loss=keras.losses.binary_crossentropy)

In [ ]:
model.summary()

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

In [ ]:
dir(ImageDataGenerator)

In [ ]:
def preprocessing_images(path):
    """
    Returns a ImageDataGenerator object.
    """
    image_data = ImageDataGenerator(
        zoom_range=0.2,
        rescale=1./255,
        horizontal_flip=True
    )
    
    images = image_data.flow_from_directory(
        directory=path,
        target_size=(224,224),
        class_mode="binary",
        shuffle=True,
        classes=["Brain Tumor","Healthy"]
    )
    
    return images

In [ ]:
train_data = preprocessing_images("data/train")
test_data = preprocessing_images("data/test")
val_data = preprocessing_images("data/val")

In [ ]:
from keras.callbacks import EarlyStopping, ModelCheckpoint, CSVLogger, ProgbarLogger, LearningRateScheduler

In [ ]:
earlystopping = EarlyStopping(
    monitor='val_loss',
    min_delta=0.1,
    patience=200,
    verbose=1,
    start_from_epoch=50,
    mode = 'min'
)

modelcheckpoint = ModelCheckpoint(
    filepath="final_model.keras",
    monitor='val_loss',
    save_best_only=True,
    verbose=1
)

logger = CSVLogger(
    filename="performance.csv",
    append=True
)

pbloader = ProgbarLogger()

In [ ]:
callbacks = [earlystopping,modelcheckpoint,logger,pbloader]

In [ ]:
model.summary()

In [ ]:
# Training the model
history = model.fit(

train_data,
steps_per_epoch=8,
epochs=1500,
verbose=1,
validation_data=val_data,
validation_steps=16,
callbacks=callbacks
)

In [ ]:
import pandas as pd
perf = pd.read_csv("/kaggle/working/performance.csv")

In [ ]:
import keras
print(keras.__version__)

In [ ]:
from keras.models import load_model

model = load_model("/kaggle/working/final_model.keras")

In [ ]:
model.summary()

In [ ]:
from keras.preprocessing.image import load_img, img_to_array

img = load_img("/kaggle/working/data/test/Healthy/Not Cancer  (1426).jpg")

arr = img_to_array(img)

arr.shape

In [ ]:
dir(ImageDataGenerator)

In [ ]:
help(ImageDataGenerator)

In [ ]:
def preprocessing_images(path):
    """
    Returns a ImageDataGenerator object.
    """
    image_data = ImageDataGenerator(
        zoom_range=0.2,
        rescale=1./255,
        horizontal_flip=True,
#         target_size=(224,224)
    )
    
    return image_data.shape

In [ ]:
image = keras.preprocessing.image.load_img("/kaggle/working/data/test/Healthy/Not Cancer  (1426).jpg")

In [ ]:
image = image.resize((224,224))
image

In [ ]:
from PIL import Image
import numpy as np

path = "/kaggle/input/brian-tumor-dataset/Brain Tumor Data Set/Brain Tumor Data Set/Brain Tumor/Cancer (1006).jpg"

image = Image.open(path)
image = image.resize((224, 224))  # Resize the image

# Convert the PIL Image to a NumPy array before reshaping
image_array = np.array(image)

# Expand dimension if grayscale (assuming it is)
if len(image_array.shape) == 2:
    image_array = image_array.reshape((224, 224, 1))

# Convert from uint8 (0-255) to float32 (0.0-1.0) for the model
image_array = image_array.astype('float32') / 255.0

# Add a new axis for batch size (even if 1)
# model.predict(image_array[np.newaxis])
threshold = 0.5  # You can adjust this based on model confidence

prediction = model.predict(image_array[np.newaxis])  # Make prediction
classification = 1 if prediction[0][0] > threshold else 0  # Apply threshold

# Print the classification result (0 - Brain Tumor, 1 - Healthy)
print("Predicted Class:", classification)


In [ ]:
model.predict(arr)

In [ ]:
model.predict(arr)

In [ ]:
df.drop('epoch', axis=1).plot()